# Risk Factor Clinical Classifier

This classifier takes patient answers to the Lung Space clinical survey and predicts the chance of cancer.

In [5]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

df = pd.read_csv('../data/lungspace/package-nlst-1193.2024-02-20/participant_d040722.csv')

# Breakdown of positive and negative sample size
print("positive: ", len(df['lung_cancer'].loc[df['lung_cancer'] == 1].to_list()), " negative: ", len(df['lung_cancer'].loc[df['lung_cancer'] == 0].to_list()))


positive:  2058  negative:  51394


/tmp/ipykernel_10406/3809338938.py:7: DtypeWarning: Columns (239,240,348) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/lungspace/package-nlst-1193.2024-02-20/participant_d040722.csv')


In [6]:
# List of features
print(df.columns.to_list())


['cen', 'dataset_version', 'elig', 'ineligible', 'pid', 'rndgroup', 'study', 'age', 'educat', 'ethnic', 'gender', 'has_bq', 'height', 'marital', 'race', 'weight', 'age_quit', 'cigar', 'cigsmok', 'pipe', 'pkyr', 'smokeage', 'smokeday', 'smokelive', 'smokework', 'smokeyr', 'last_screen_studyyr', 'num_screens', 'scr_days0', 'scr_days1', 'scr_days2', 'scr_group', 'scr_iso0', 'scr_iso1', 'scr_iso2', 'scr_lat0', 'scr_lat1', 'scr_lat2', 'scr_res0', 'scr_res1', 'scr_res2', 'biop0', 'biop1', 'biop2', 'bioplc', 'invas0', 'invas1', 'invas2', 'invaslc', 'medcomp0', 'medcomp1', 'medcomp2', 'medcomplc', 'mra_stat0', 'mra_stat1', 'mra_stat2', 'no_proc_reas0', 'no_proc_reas1', 'no_proc_reas2', 'proc0', 'proc1', 'proc2', 'proclc', 'can_scr', 'canc_free_days', 'canc_rpt_link', 'canc_rpt_source', 'cancyr', 'candx_days', 'conflc', 'de_grade', 'de_stag', 'de_stag_7thed', 'de_type', 'histology_cat', 'lesionsize', 'loccar', 'loclhil', 'loclin', 'locllow', 'loclmsb', 'loclup', 'locmed', 'locoth', 'locrhil', '

In [12]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(gamma=2, C=1, random_state=42),
    GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
    DecisionTreeClassifier(max_depth=5, random_state=42),
    RandomForestClassifier(
        max_depth=5, n_estimators=10, max_features=1, random_state=42
    ),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    AdaBoostClassifier(algorithm="SAMME", random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

df = df[['age', 'weight', 'height', "cigar","cigsmok","pipe","pkyr","smokeage","smokeday","smokelive","smokework","smokeyr","last_screen_studyyr","num_screens", "lung_cancer"]]
df.dropna()

X = df[['age', 'weight', 'height', "cigar","cigsmok","pipe","pkyr","smokeage","smokeday","smokelive","smokework","smokeyr","last_screen_studyyr","num_screens"]]
y = df[['lung_cancer']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42
)

print(X_train)
print(X_test)
print(y_train)
print(y_test)


       age  weight  height  cigar  cigsmok  pipe    pkyr  smokeage  smokeday  \
32200   63   138.0    62.0    0.0        1   0.0   43.00      18.0        20   
40273   63   265.0    75.0    1.0        0   1.0   71.75      10.0        35   
36492   70   127.0    67.0    1.0        1   1.0   43.00      27.0        20   
21588   55   210.0    66.0    0.0        0   0.0   33.00      20.0        20   
10580   58   145.0    67.0    1.0        0   1.0   37.50      20.0        25   
...    ...     ...     ...    ...      ...   ...     ...       ...       ...   
11284   61   170.0    69.0    1.0        1   0.0   69.00      15.0        30   
44732   61   173.0    67.0    1.0        1   1.0   33.00      17.0        15   
38158   58   172.0    67.0    1.0        1   1.0   87.50      18.0        50   
860     67   170.0    74.0    1.0        0   1.0  117.50      13.0        50   
15795   56   168.0    64.0    0.0        0   1.0   54.00      19.0        40   

       smokelive  smokework  smokeyr  l

In [14]:
for name, clf in zip(names, classifiers):
  clf = make_pipeline(StandardScaler(), clf)
  clf.fit(X_train, y_train)
  score = clf.score(X_test, y_test)


InvalidIndexError: (slice(None, None, None), 0)